In [3]:
# Import target.npy and train.npy
# Train a model using train.npy
# Test the model using target.npy

import numpy as np
from sklearn.model_selection import train_test_split
# Import the data
train = np.load('dataset/train.npy')
target = np.load('dataset/target.npy')

In [4]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import plot_model
from sklearn.model_selection import train_test_split
import keras_tuner as kt

def NN_RMSLE(y_actual, y_pred):
    return tf.sqrt(tf.reduce_mean(tf.square(tf.math.log1p(y_pred) - tf.math.log1p(y_actual))))

def build_model(hp):
    model = Sequential()
    model.add(Dense(units=hp.Int('units1', min_value=32, max_value=512, step=32),
                    activation='relu', input_shape=(train_xx.shape[1],)))
    model.add(Dropout(rate=hp.Float('dropout1', min_value=0.0, max_value=0.5, step=0.1)))
    model.add(Dense(units=hp.Int('units2', min_value=32, max_value=256, step=32), activation='relu'))
    model.add(Dropout(rate=hp.Float('dropout2', min_value=0.0, max_value=0.5, step=0.1)))
    model.add(Dense(units=hp.Int('units3', min_value=32, max_value=128, step=32), activation='relu'))
    model.add(Dropout(rate=hp.Float('dropout3', min_value=0.0, max_value=0.5, step=0.1)))
    model.add(Dense(1, activation='linear'))

    model.compile(optimizer='adam', loss=NN_RMSLE)
    return model

# Load your data
# train, target = # Assume train and target are loaded appropriately
train_xx, val_xx, train_yy, val_yy = train_test_split(train, target, test_size=0.2, random_state=42)

# Setup the tuner
tuner = kt.RandomSearch(
    build_model,
    objective='val_loss',
    max_trials=3,  # How many model configurations would you like to try?
    executions_per_trial=1,  # How many trials per model configuration?
    directory='keras_tuner_dir',
    project_name='keras_tuner_demo'
)

earlyStop = EarlyStopping(monitor='val_loss', mode='min', patience=3)
tuner.search(train_xx, train_yy, epochs=15, batch_size=2048, validation_data=(val_xx, val_yy), callbacks=[earlyStop])

# Get the optimal hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"""
The optimal number of units in the first dense layer is {best_hps.get('units1')} 
and the optimal dropout rate in the first dropout layer is {best_hps.get('dropout1')}.
The optimal number of units in the second dense layer is {best_hps.get('units2')} 
and the optimal dropout rate in the second dropout layer is {best_hps.get('dropout2')}.
The optimal number of units in the third dense layer is {best_hps.get('units3')} 
and the optimal dropout rate in the third dropout layer is {best_hps.get('dropout3')}.
""")

# Build the model with the optimal hyperparameters and train it on the data
model = tuner.hypermodel.build(best_hps)
model.fit(train_xx, train_yy, epochs=15, batch_size=2048, validation_data=(val_xx, val_yy), callbacks=[earlyStop])

model.save('optimized_model.keras')
model.summary()
plot_model(model, to_file='model.png', show_shapes=True)


Reloading Tuner from keras_tuner_dir\keras_tuner_demo\tuner0.json

Search: Running Trial #3

Value             |Best Value So Far |Hyperparameter
384               |288               |units1
0.3               |0                 |dropout1
192               |224               |units2
0.2               |0.2               |dropout2
64                |96                |units3
0.2               |0                 |dropout3

Epoch 1/15
7755/7755 ━━━━━━━━━━━━━━━━━━━━ 123s 16ms/step - loss: nan - val_loss: nan
Epoch 2/15
7755/7755 ━━━━━━━━━━━━━━━━━━━━ 68s 9ms/step - loss: nan - val_loss: nan
Epoch 3/15
7755/7755 ━━━━━━━━━━━━━━━━━━━━ 125s 16ms/step - loss: nan - val_loss: nan


RuntimeError: Number of consecutive failures exceeded the limit of 3.


: 